# ATIAM - Fundamentals project
## Sequence alignment
Author: Philippe Esling (esling@ircam.fr)

This notebook defines the overall exercise for the ATIAM data structure course.

**General indications**
 - You should use this as a baseline notebook
 - You are authorized to define other files for functions
 - Write a (small) report document (PDF) explaining your approach
 - All your files should be packed in a zip file named
     [ATIAM][FpA2024]FirstName_LastName.zip

**Deadline**: 14/11/2024 @ 23:59

In [11]:
# Define mode (keep it on False, this is just for my generative part)
DEV_MODE=False
# Basic set of imports (here you can see if everything passes)
import os
import pickle
import string

#%% Here collect the whole set of tracks
if DEV_MODE:
    # Define MIDI extension
    midExt = ['mid', 'midi', 'MID', 'MIDI']
    # Root directory
    root = '/Users/esling/Coding/learning/orchestral-db/data'
    database = {}
    composers = []
    composers_tracks = {}
    tracks = []
    # List composers first
    for item in os.listdir(root):
        if os.path.isdir(os.path.join(root, item)):
            composers.append(item)
    print('Found ' + str(len(composers)) + ' composers.')
    prev_letter = ''
    # now parse tracks
    for comp in sorted(composers):
        # Print advance
        if (comp[0] != prev_letter):
            prev_letter = comp[0]
            print('   - Composers starting with ' + prev_letter)
        # Check each sub-folder
        for item in os.listdir(os.path.join(root, comp)):
            cur_path = os.path.join(os.path.join(root, comp), item)
            if os.path.isdir(cur_path):
                for files in os.listdir(cur_path):
                    if (os.path.splitext(files)[1][1:] in midExt):
                        tracks.append(item)
                        if comp in composers_tracks.keys():
                            composers_tracks[comp].append(item)
                        else:
                            composers_tracks[comp] = [item]
    print('Found ' + str(len(tracks)) + ' tracks.')
    midi_database = {'composers':composers, 'composers_tracks':composers_tracks}
    pickle.dump(midi_database, open("atiam-fpa.pkl", "wb"))
else:
    midi_database = pickle.load(open("atiam-fpa.pkl", "rb"))
    composers = midi_database['composers']
    composers_tracks = midi_database['composers_tracks']

## Part 1 - Exploration

**Exploring a track collections (text dictionnaries) and playing with MIDI**

In this part, we will start easy by looking at a collection of tracks.
The set of classical music pieces is provided in the _atiam-fpa.pkl_ file, which
is already loaded at this point of the script and contain two structures
- `composers`         = Array of all composers in the database
- `composers_tracks`  = Hashtable of tracks for a given composer
    
Some examples of the content of these structures
```python
# Names of composers
composers[23] => 'Abela, Placido'
composers[1210]  => 'Beethoven, Ludwig van'
# List of tracks for each composer
composers_tracks['Abela, Placido'] => ['Ave Maria(Meditation on Prelude No. 1 by J.S.Bach)']
composers_tracks['Beethoven, Ludwig van'] => ['Drinking Song', 'Sonatine No. 3 for Mandolin and Piano', ...]
# Retrieve the first track of 
composers_tracks['Beethoven, Ludwig van'][0] => 'Drinking Song'
```

In [ ]:
# Here an example: print all composers with more than 10 tracks
for composer,tracks in sorted(composers_tracks.items()):
    if (len(tracks) >= 10):
        print(composer + ' : ' + str(len(tracks)) + ' tracks.')

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 1.1 [1 pt] - Implement array sorting 

> 1. Reimplement the quicksort algorithm.
> 2. Show your results on different types of array

</div>

In [ ]:
def my_sort(array):
    sorted_array = []
    ################
    # YOUR CODE HERE
    ################
    return sorted_array

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 1.2 [1 pt] - Sort a composer array

> 1. Use your own algorithm to sort the collection of composers by decreasing number of tracks

</div>

In [ ]:
################
# YOUR CODE HERE
################

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 1.3 [1 pt] - Extending sort

> 1. Extend your sorting procedure, to sort all tracks from all composers alphabetically 
> 2. Evaluate the running time of your algorithm

</div>

In [ ]:
################
# YOUR CODE HERE
################

## Introducing MIDI 

In addition to our composition pickle file, you can find some example MIDI files in the `atiam-fpa/` folder. In this part, we are going to *import and plot* the different MIDI files. We recommend to use the `pretty_midi` library 
```bash
pip install pretty_midi
```

Based on the provided MIDI files (random subset of Beethoven tracks), you will have to import, plot and compare the different files.

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 1.4 [1 pt] - Basic operations

> 1. Import random tracks from the folder
> 2. Plot (display) the content, either as piano rolls or score.

</div>


In [ ]:
################
# YOUR CODE HERE
################

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 1.5 [1 pt] - Statistics operations

> 1. Write a function counting the number of notes played in a given MIDI file.
> 2. Sort the set of MIDI files based on the number of notes.

</div>

In [ ]:
################
# YOUR CODE HERE
################

## Part 2 - Symbolic alignments

In this part, we will use our knowledge on computer structures to solve a very well-known problem of **string alignement**. Hence, this part is split between
1. Implementing the string alignment algorithm
2. Trying to apply this to the collection of **names** of classical music pieces
3. Developing your own more adapted procedure to have a matching inside large set
  
The set of classical music pieces is provided in the `atiam-fpa.pkl` file, which is already loaded at this point of the script and contain two structures
- `composers`         = Array of all composers in the database
- `composers_tracks`  = Hashtable of tracks for a given composer
    
Some examples of the content of these structures

```python
composers[23] => 'Abela, Placido'
composers[1210]  => 'Beethoven, Ludwig van'

composers_tracks['Abela, Placido'] => ['Ave Maria(Meditation on Prelude No. 1 by J.S.Bach)']
composers_tracks['Beethoven, Ludwig van'] => ['"Ode to Joy"  (Arrang.)', '10 National Airs with Variations, Op.107 ', ...]

composers_tracks['Beethoven, Ludwig van'][0] => '"Ode to Joy"  (Arrang.)'
```

### Needleman-Wunsch algorithm

In order to obtain a string alignment, we are going to rely on the [Needleman-Wunsch (NW)](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm) algorithm. You can find the definition of the basic NW algorithm on Wikipedia:

https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm

In order to check your own implementation, you can use the `minineedle` library that provides different versions of the NW algorithm.

In [ ]:
from minineedle import needle, core
seq1 = "CEELECANTH"; seq2 = "PELICAN"
alignment: needle.NeedlemanWunsch[str] = needle.NeedlemanWunsch(seq1, seq2)
# Change the matrix and run
alignment.change_matrix(core.ScoreMatrix(match=4, miss=-4, gap=-2))
alignment.align()
# Print the sequences aligned
print(alignment)

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 2.1 [3 pts] - Alignment algorithm

> 1. Write the basic NW algorithm.
> 2. In this first version, we will be implementing the _basic_ gap costs
> 3. Remember to rely on a user-defined matrix for symbols distance

</div>

In [16]:
def my_needleman_simple(str1, str2, matrix='atiam-fpa_alpha.dist', gap_open=-5, gap_extend=-5):
    score = 0
    ################
    # YOUR CODE HERE
    ################
    return ('', '', score)

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 2.2 [1 pt] - Application to musical track names

> 1. For each track of a composer, compare to all remaining tracks of the same composer
> 2. Establish a cut criterion (what is the relevant similarity level ?) to only print relevant matches
> 3. Propose a set of matching tracks and save it through Pickle

</div>

In [ ]:
################
# YOUR CODE HERE
################

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 2.3 [2 pts] - Advanced properties extensions

> 1. Try to compare a given track to all tracks of all composers (full database)
> 2. As the time taken is untractable (computational explosion), propose a method to avoid such a huge amount of computation
> 3. Propose a set of matching tracks and save it through Pickle

</div>

In [ ]:
################
# YOUR CODE HERE
################

## Part 3 - Musically-informed track names alignment

You might have seen from the previous results that
- Purely string matching on classical music names is not the best approach
- This mostly comes from the fact that the importance of symbols is not the same
- For instance 
  * "Symphony for orchestra in D minor"
  * "Symphony for orchestra in E minor"
Looks extremely close but the key is the most important symbol
  
Another flaw in our approach is that the NW algorithm treats all gaps equivalently. Hence, it can put lots of small gaps everywhere. Regarding alignement, it would be better to have long coherent gaps rather than small ones. This is handled by a mecanism known as _affine gap penalty_ which separates the costs of either _opening_ or _extending_ a gap. This is known as the Gotoh algorithm, which can be found here :

http://helios.mi.parisdescartes.fr/~lomn/Cours/BI/Material2019/gap-penalty-gotoh.pdf

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 3.1 [2 pts] - Extension to true musical names matching

> 1. Propose a new name matching algorithm adapted to classical music piece names
- Can be based on a rule-based system
- Can be a pre-processing for symbol finding and then an adapted weight matrix
- Can be a local-alignement procedure
(These are only given as indicative ideas ...)
> 2. Implement this new comparison procedure adapted to classical music piece names
> 3. Re-run your previous results (Q-2.2 and Q-2.3) with this procedure

</div>

In [ ]:
################
# YOUR CODE HERE
################

# Example of creating a dummy matrix
if DEV_MODE:
    dist = open('atiam-fpa_alpha.dist', 'w')
    dist.write('   ')
    for m1 in string.ascii_uppercase:
        dist.write(m1)
        if (m1 < 'Z'):
            dist.write('  ')
    dist.write('\n')
    for m1 in string.ascii_uppercase:
        dist.write(m1 + '  ')
        for m2 in string.ascii_uppercase:
            if (m2 == m1):
                dist.write('5  ')
            else:
                dist.write('-3  ')
        dist.write('\n')
    dist.close()

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 3.2 [3 pts] - Extending the NW algorithm

> 1. Add the affine gap penalty to your original NW algorithm (using the Gotoh algorithm)
> 2. Verify your code by using the provided reference version

</div>

In [ ]:
################
# YOUR CODE HERE
################

## Part 4 - Musical alignments between MIDI files

Interestingly the problem of string alignment can be extended to the more global  problem of aligning any series of symbolic information (vectors). Therefore, we can see that the natural extension of this problem is to align any sequence of symbolic information.

This definition matches very neatly to the alignement of two musical scores  that can then be used as symbolic similarity between music, or score following. However, this requires several key enhancements to the previous approach. Furthermore, MIDI files gathered on the web are usually of poor quality and require to be checked. Hence, here you will
1. Learn how to read and watch MIDI files
2. Explore their properties to perform some quality checking
3. Extend alignment to symbolic score alignement
    
To fasten the pace of your musical analysis, we will rely on the excellent `Music21` library, which provides all sorts of musicological analysis and properties over symbolic scores. You will need to really perform this part to go and [read the documentation of this library online](http://web.mit.edu/music21/doc/). We provide in the next cell a few example of using this library

In [30]:
import math
import numpy as np
from music21 import converter

def get_start_time(el,measure_offset,quantization):
    if (el.offset is not None) and (el.measureNumber in measure_offset):
        return int(math.ceil(((measure_offset[el.measureNumber] or 0) + el.offset)*quantization))
    # Else, no time defined for this element and the functino return None

def get_end_time(el,measure_offset,quantization):
    if (el.offset is not None) and (el.measureNumber in measure_offset):
        return int(math.ceil(((measure_offset[el.measureNumber] or 0) + el.offset + el.duration.quarterLength)*quantization))
    # Else, no time defined for this element and the functino return None
    
def get_pianoroll_part(part,quantization):
    # Get the measure offsets
    measure_offset = {None:0}
    for el in part.recurse(classFilter=('Measure')):
        measure_offset[el.measureNumber] = el.offset
    # Get the duration of the part
    duration_max = 0
    for el in part.recurse(classFilter=('Note','Rest')):
        t_end = get_end_time(el,measure_offset,quantization)
        if(t_end>duration_max):
            duration_max=t_end
    # Get the pitch and offset+duration
    piano_roll_part = np.zeros((128,math.ceil(duration_max)))
    for this_note in part.recurse(classFilter=('Note')):
        note_start = get_start_time(this_note,measure_offset,quantization)
        note_end = get_end_time(this_note,measure_offset,quantization)
        piano_roll_part[this_note.pitch.midi,note_start:note_end] = 1
    return piano_roll_part

# Here we provide a MIDI import function
def importMIDI(f):
    piece = converter.parse(f)
    all_parts = {}
    for part in piece.parts:
        try:
            track_name = part[0].bestName()
        except AttributeError:
            track_name = 'None'
        cur_part = get_pianoroll_part(part, 16);
        if (cur_part.shape[1] > 0):
            all_parts[track_name] = cur_part;
    print('Returning')
    return piece, all_parts

The Music21 library propose a lot of properties directly on the piece element, but we also provide separately a dictionary containing for each part a matrix representation (pianoroll) of the corresponding notes (without dynamics). To answer the following questions, you can rely on the [Music21 documentation](http://web.mit.edu/music21/doc/)

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 4.1 [3 pts] - Exploring MIDI properties

> 1. Explore various musicology properties proposed by the library
> 2. Check which could be used to assess the quality of MIDI files

</div>

In [ ]:
# Here a few properties that can be plotted ...
piece, all_parts = importMIDI("atiam-fpa/beethoven_0.mid")
piece.plot('scatter', 'quarterLength', 'pitch')
piece.plot('scatterweighted', 'pitch', 'quarterLength')
piece.plot('histogram', 'pitchClass')

One of the most pervasive problem with MIDI scores is that a large part of the files that you can find on the internet are of rather low quality. Based on your exploration in the previous questions and your own intuition,

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 4.2 [4 pts] - Automatic evaluation of a MIDI file quality

> 1. Propose an automatic procedure that could evaluate the quality of a MIDI file.
> 2. Test how this could be used on a whole set of files

</div>

In [35]:

################
# YOUR CODE HERE
################

As explained earlier, our alignment algorithm can work with any set of symbols, which of course include even complex scores. The whole trick here is to see that the "distance matrix" previously used could simply be replaced by a "distance function", which can represent the similarity between any elements.

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Q 4.3 [4 pts] - Extending your alignment algorithm to MIDI scores

> 1. Propose a fit distance measures between two slices of pianorolls
> 2. Modify your previous algorithm so that it can use your distance
> 3. Modify the algorithm so that it can work with MIDI files
> 4. Apply your algorithm to sets of MIDI files

</div>

In [36]:

################
# YOUR CODE HERE
################